In [2]:
import utils
from PIL import Image
from facenet_pytorch import MTCNN
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from keras.models import load_model
from matplotlib.patches import Circle
import cv2
import numpy as np
from sklearn.preprocessing import Normalizer
from scipy.spatial.distance import cosine
import os
import datetime
import xlwt
import xlrd
from xlwt import Workbook 
from xlutils.copy import copy
import time

In [3]:
device='cuda'
encoder_model = './model/facenet_keras.h5'

In [4]:
detector=MTCNN(keep_all=True, device=device)
face_encoder = load_model(encoder_model,compile=False)
people_dir = './people'
encoding_dict = dict()

In [5]:
classNames = []
with open('coco.names','r') as f:
    classNames = f.read().splitlines()
print(classNames)

['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'hair brush']


In [6]:
weightsPath = "frozen_inference_graph.pb"
configPath = "ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

<dnn_Model 0x7fc55ede6270>

In [7]:
def get_encode(face_encoder, face, size):
    face = normalize(face)
    face = cv2.resize(face, size)
    encode = face_encoder.predict(np.expand_dims(face, axis=0))[0]
    return encode

In [8]:
def get_face(img, box):
    box=[int(i) for i in box]
    x1, y1, x2, y2 = box
    face = img[y1:y2, x1:x2]
    return face, (x1, y1), (x2, y2)

In [9]:
def normalize(img):
    mean, std = img.mean(), img.std()
    return (img - mean) / std

In [10]:
l2_normalizer = Normalizer('l2')

In [11]:
def export(a):
    ct = datetime.datetime.now()
    
    ws = xlrd.open_workbook('names_list.xls',formatting_info=True)
    sheet = ws.sheet_by_index(0)
    wb=copy(ws)
    names_sheet=wb.get_sheet(0)
    row_s=sheet.nrows
    col_s=sheet.ncols
    
    names_sheet.write(0,col_s,str(ct.year)+"_"+str(ct.month)+"_"+str(ct.day)+"_"+str(ct.hour))
    
    for i in range(1,row_s):   
        
        if sheet.cell_value(i,0) in a:
            names_sheet.write(i, col_s,"P")
                
        else:
                
            names_sheet.write(i, col_s,"A")
    wb.save("names_list.xls") 

In [12]:
def mark_attendance(a):
    

    ct = datetime.datetime.now()
    
    


    workbook = xlwt.Workbook()  

    sheet = workbook.add_sheet(str(ct.year)+"_"+str(ct.month)+"_"+str(ct.day)) 
    sheet.write(0,0,"Name")


    row = 1
    col = 0
    if len(a)>0:
        for person_name in os.listdir(people_dir):
            print(person_name)
            print(a)

            for x in range(0,len(a)):
                if str(person_name) in a:
                    sheet.write(row, col,     str(a[x]))
                    sheet.write(row,col+1,"P")
                    
                else:
                    sheet.write(row, col,     str(a[x]),)
                    sheet.write(row,col+1,"A")
                row+=1
         
            
  
        workbook.save("sample_class_1.xls") 
    else:
        sheet.write(1,0,"No one is present")
        workbook.save("sample_class_1.xls") 

In [13]:
dimensions=(160,160)

In [14]:
for person_name in os.listdir(people_dir):
    person_dir = os.path.join(people_dir, person_name)
    encodes = []
    for img_name in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_name)
        img = cv2.imread(img_path)

        bounding_boxes, conf= detector.detect(img, landmarks=False)
#         results, probs= detector.detect(img)
        
        
        
        if bounding_boxes !=[]:
            try:
                face, _, _ = get_face(img, bounding_boxes[0])
                face = normalize(face)
                face = cv2.resize(face,dimensions)
                encode = face_encoder.predict(np.expand_dims(face, axis=0))[0]
                encodes.append(encode)
            except Exception as e: 
                print(str(e))
                print(person_name+" "+img_name)
                
    if encodes:
        encode = np.sum(encodes, axis=0)
        encode = l2_normalizer.transform(np.expand_dims(encode, axis=0))[0]
        encoding_dict[person_name] = encode  

/home/aditya/.local/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  del sys.path[0]
/home/aditya/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: Mean of empty slice.
  
/home/aditya/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/aditya/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/aditya/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/aditya/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


OpenCV(4.4.0) /tmp/pip-req-build-njn2fp78/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Aditya 5.jpeg
'NoneType' object is not subscriptable
Aditya 3.jpeg


/home/aditya/.local/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  del sys.path[0]
/home/aditya/.local/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  del sys.path[0]


In [15]:
encoding_dict

{'Aditya': array([-0.03904701, -0.10442009,  0.14776303,  0.2500403 ,  0.04244868,
         0.11182255, -0.00874341, -0.11569592,  0.07027978,  0.08836868,
        -0.17368801, -0.02335205,  0.01081557, -0.04059359, -0.11617737,
        -0.07648676,  0.08868869,  0.06259269,  0.03583365, -0.05488308,
        -0.08766231,  0.01579584,  0.06335662, -0.04887755, -0.02077567,
        -0.03340274, -0.00634047, -0.06535081, -0.05890035, -0.03134989,
         0.09241497, -0.10865742, -0.04815555, -0.09910399,  0.07510326,
        -0.07544328,  0.00995452,  0.12201301,  0.10748708,  0.00824658,
         0.03075092,  0.06478517,  0.11489124,  0.00865825,  0.13696599,
         0.04641121, -0.07612006,  0.14606093,  0.06626271,  0.11691636,
         0.06513786, -0.04402125,  0.01429342, -0.0052444 , -0.00871318,
         0.04938497,  0.09431054, -0.10523877, -0.01095662,  0.0079315 ,
         0.02879027, -0.12766616,  0.03041083,  0.09087356, -0.02884291,
        -0.01036598, -0.03691437,  0.0265

In [16]:
recognition_t=0.6
confidence_t=0.99
thres = 0.5 # Threshold to detect object
nms_threshold = 0.2 #(0.1 to 1) 1 means no suppress , 0.1 means high suppress
font = cv2.FONT_HERSHEY_PLAIN
#font = cv2.FONT_HERSHEY_COMPLEX
Colors = np.random.uniform(0, 255, size=(len(classNames), 3))

In [17]:
video =cv2.VideoCapture(0)
present_candidates=[]

fps_start_time = datetime.datetime.now()
fps = 0
total_frames = 0

while True:
    check,frame=video.read()
    total_frames = total_frames + 1
    
    classIds, confs, bbox = net.detect(frame,confThreshold=thres)
    bbox = list(bbox)
    confs = list(np.array(confs).reshape(1,-1)[0])
    confs = list(map(float,confs))
    
    indices = cv2.dnn.NMSBoxes(bbox,confs,thres,nms_threshold)
    
    if len(classIds) != 0:
        
        for i in indices:
            i = i[0]
            box = bbox[i]
            confidence = str(round(confs[i],2))
            color = Colors[classIds[i][0]-1]
            x,y,w,h = box[0],box[1],box[2],box[3]
            cv2.rectangle(frame, (x,y), (x+w,y+h), color, thickness=2)
            cv2.putText(frame, classNames[classIds[i][0]-1]+" "+confidence,(x+10,y+20),
                        font,1,color,2)

    bounding_boxes, conf, landmarks = detector.detect(frame, landmarks=True)
    
    if bounding_boxes is not None: 
        for i in range(len(bounding_boxes)):
            face, pt_1, pt_2 = get_face(frame,bounding_boxes[i])
            cv2.rectangle(frame, pt_1 , pt_2, (0, 255, 0), 2)
            try:
                encode = get_encode(face_encoder, face,(160,160))
                encode = l2_normalizer.transform(encode.reshape(1, -1))[0]
                name = 'unknown'


                distance = float("inf")


                for (db_name, db_enc) in encoding_dict.items():

                    dist = cosine(db_enc, encode)

                    if dist < recognition_t and dist < distance:

                        name = db_name
                        distance = dist
                        if name not in present_candidates:
                            present_candidates.append(name)


                if name == 'unknown':
                    cv2.rectangle(frame, pt_1, pt_2, (0, 0, 255), 2)
                    cv2.putText(frame,name, pt_1, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)

                else:

                    cv2.rectangle(frame, pt_1 , pt_2, (0, 255, 0), 2)
                    cv2.putText(frame,name + f'__{distance:.2f}', (pt_1[0], pt_1[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0, 200, 200), 2)
                    
            except Exception as e: 
                print(str(e))

            
            
    fps_end_time = datetime.datetime.now()
    time_diff = fps_end_time - fps_start_time
    if time_diff.seconds == 0:
        fps = 0.0
    else:
        fps = (total_frames / time_diff.seconds)

    fps_text = "FPS: {:.2f}".format(fps)

    cv2.putText(frame, fps_text, (5, 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)

    cv2.imshow('frame',frame)  
    if cv2.waitKey(5) & 0xFF == ord('q'):
          break
            
            
            
video.release()
cv2.destroyAllWindows()